In [ ]:
!pip -q install huggingface chromadb transformers langchain

In [ ]:
!pip install InstructorEmbedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embedding class in LangChain exposes two methods: embed_documents and embed_query. The largest difference is that these two methods have different interfaces: one works over multiple documents, while the other works over a single document. Besides this, another reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

In [ ]:
hfEmbed = HuggingFaceEmbeddings()

In [ ]:
hfEmbed.model_name

'sentence-transformers/all-mpnet-base-v2'

In [ ]:
hfInstructEmbed = HuggingFaceInstructEmbeddings(
 query_instruction="Represent the query for retrieval: "   
)

In [ ]:
hfInstructEmbed.model_name

'hkunlp/instructor-large'

In [ ]:
text = "This is a test document."

In [ ]:
query_result = hfEmbed.embed_query(text)

In [ ]:
query_result

In [ ]:
doc_result = hfEmbed.embed_documents([text])

In [ ]:
doc_result

In [ ]:
instructor_result = hfInstructEmbed.embed_query(text)

In [ ]:
instructor_result

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

In [ ]:
spaceLoad = TextLoader('/content/database_tutor.txt')

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
documents = spaceLoad.load()

In [ ]:
documents

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
from langchain.vectorstores import Chroma

data_space = Chroma.from_documents(texts,hfEmbed)

In [ ]:
#using data_space as retriever

space_retriever = data_space.as_retriever()

In [ ]:
#I am betting on using the recent dolly

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [ ]:
!pip install configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import configparser

api_reader = configparser.ConfigParser()
api_reader.read_file(open('/content/apidata.config'))

In [ ]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = api_reader["OPENAI"]["HFREAD"]

In [ ]:
llm=HuggingFaceHub(repo_id="EleutherAI/gpt-j-6B",
                   model_kwargs={"temperature":0.1, 
                                 "max_length":490})

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff", 
                                 retriever=space_retriever)

In [ ]:
qa.run("Summmarize the text")

ValueError: ignored